In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=9e915c4bf1fe4e110ba5b940e844ca597d5bc526bdfc4627bec57565464d8427
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import pyspark

sc = pyspark.SparkContext()
sqlContext = pyspark.sql.SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive' | grep parquet

20-05-2023-data.parquet


In [5]:
parquet_file_path = '/content/drive/My Drive/20-05-2023-data.parquet'
df = sqlContext.read.parquet(parquet_file_path)

In [10]:
df.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|           Green Day|Logged In|  Herbert|     M|            1|    Long|547.44771| free|Riverside-San Ber...|   PUT|NextSong|1633337071769|      226|Jesus Of Suburbia...|   200|1653629820769|"Mozilla/5.0 (Mac...|    33|
|Emil Gilels/Orche...|Logged In|    Arjun|     M|           18|   Reyes|375.19628| paid|    Fayetteville, NC|   PUT|Next

In [6]:
df.count()

8484

In [19]:
# tìm top 5 bài hát (song) đang được nghe nhiều nhất
# tính bằng tổng thời gian bài hát đó được nghe (length)
song_length_df = df[["song", "length"]].groupby('song').sum()
song_length_df.sort(['sum(length)'], ascending=False).head(5)

[Row(song='Sehr kosmisch', sum(length)=12459.77269),
 Row(song="You're The One", sum(length)=8136.4482000000025),
 Row(song='Undo', sum(length)=6274.39518),
 Row(song='240 Years Before Your Time', sum(length)=5601.0692),
 Row(song="I CAN'T GET STARTED", sum(length)=4971.3587)]

In [115]:
# phân tích thời gian sử dụng app vào cuôi tuần theo giới tính
# từ ts (timstamp) và gender
# output sẽ có dạng
# gender    total_time_in_week_day  total_time_in_weekend
# male      12345                   6789
# female    24659                   65214
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType, IntegerType, BooleanType

@udf(BooleanType())
def is_week_day(ts):
    print(ts)
    ts_int = int (ts / 1000)
    return datetime.fromtimestamp(ts_int).weekday() < 5

week_day_time = df[['gender', 'length', 'ts']] \
    .withColumn('is_week_day', is_week_day(df.ts.cast('long')))
week_day_time.createOrReplaceTempView("week_day_time")
sqlContext.sql("select gender, is_week_day, sum(length) from week_day_time where gender != null group by is_week_day, gender").show()



+------+-----------+------------------+
|gender|is_week_day|       sum(length)|
+------+-----------+------------------+
|     F|       true|320953.91722000006|
|     M|      false|247653.92401999992|
|     M|       true|1123211.9636799989|
|     F|      false| 75279.82385999999|
|  null|       true|               0.0|
|  null|      false|               0.0|
+------+-----------+------------------+

